# Collaborative Filtering of Movies
## MSDS 7349 - Section 401
## Project 3

[Data Science @ Southern Methodist University](https://datascience.smu.edu/)

# Table of Contents
* [Team Members](#Team-Members)
* [Import Dataset](#Import)
* [Business Understanding](#Business-Understanding)
* [Data Meaning Type](#Data-Meaning-Type)
* [Data Quality](#Data-Quality)
* [Simple Statistics](#Simple-Statistics)
* [Visualize Attributes](#Visualize-Attributes)
* [Explore Joint Attributes](#Explore-Joint-Attributes)
* [Explore Attributes and Class](#Explore-Attributes-and-Class)
* [New Features](#New-Features)
* [Exceptional Work](#Exceptional-Work)
* [References](#References)

# <a name="Team-Members"></a>Team Members
* [Jostein Barry-Straume](https://github.com/josteinstraume)
* [Kevin Cannon](https://github.com/kcannon2)
* [Ernesto Carrera Ruvalcaba](https://github.com/ecarrerasmu)
* [Adam Tschannen](https://github.com/adamtschannen)

# <a name="Import"></a>Import Dataset

In [17]:
%%capture
# Source:
# https://github.com/eclarson/DataMiningNotebooks/blob/master/13.%20Recommendation%20Systems.ipynb
# Supress output - https://stackoverflow.com/questions/23692950/how-do-you-suppress-output-in-ipython-notebook
import graphlab as gl;

filepath = '../data/ml-latest-small/ratings.csv';
ratings = gl.SFrame.read_csv(filepath, column_type_hints = {"rating":int});
movies = gl.SFrame.read_csv('../data/ml-latest-small/movies.csv');
data = ratings.join(movies, on = 'movieId', how = 'left');

# <a name="Business-Understanding"></a>Business Understanding
*10 points] Describe the purpose of the data set you selected (i.e., why was this data collected in the first place?). How will you measure the effectiveness of a good algorithm? Why does your chosen validation method make sense for this specific dataset and the stakeholders needs?*

In [ ]:
# Jostein insert work here

# <a name="Data-Meaning-Type"></a>Data Meaning Type
*[10 points] Describe the meaning and type of data (scale, values, etc.) for each
attribute in the data file. Verify data quality: Are there missing values? Duplicate data?
Outliers? Are those mistakes? How do you deal with these problems?*

>"This dataset (ml-20m) describes 5-star rating and free-text tagging activity from MovieLens, a movie recommendation service. It contains 20000263 ratings and 465564 tag applications across 27278 movies. These data were created by 138493 users between January 09, 1995 and March 31, 2015. This dataset was generated on October 17, 2016.

>Users were selected at random for inclusion and IDS have anonumized. All selected users had rated at least 20 movies. No demographic information is included. Each user is represented by an id, and no other information is provided."

>The Movie IDs all contain "at least one rating or tag are included in the dataset."

>The generes of movie are: Action,	Animation,	Comedy,	Drama,	Documentary, and	Romance	Short.

In [16]:
movies.head()

movieId,title,genres
1,Toy Story (1995),Adventure|Animation|Children|Comedy|Fantasy ...
2,Jumanji (1995),Adventure|Children|Fantasy ...
3,Grumpier Old Men (1995),Comedy|Romance
4,Waiting to Exhale (1995),Comedy|Drama|Romance
5,Father of the Bride PartII (1995) ...,Comedy
6,Heat (1995),Action|Crime|Thriller
7,Sabrina (1995),Comedy|Romance
8,Tom and Huck (1995),Adventure|Children
9,Sudden Death (1995),Action
10,GoldenEye (1995),Action|Adventure|Thriller


*[10 points] Visualize the any important attributes appropriately. Important: Provide an
interpretation for any charts or graphs.*

In [18]:
# Ernesto insert work here

# <a name="Modeling-and-Evaluation"></a>Modeling and Evaluation
*Different tasks will require different evaluation methods. Be as thorough as possible when analyzing the data you have chosen and use visualizations of the results to explain the performance and expected outcomes whenever possible. Guide the reader through your analysis with plenty of discussion of the results. Each option is broken down by:*

*[10 Points] Train and adjust parameters*

*[10 Points] Evaluate and Compare*

*[10 Points] Visualize Results*

*[20 Points] Summarize the Ramifications*

# <a name="Deployment"></a>Deployment
*Be critical of your performance and tell the reader how you current model might be usable by other parties. Did you achieve your goals? If not, can you reign in the utility of your modeling?*
* How useful is your model for interested parties (i.e., the companies or organizations that might want to use it)?
* How would your deploy your model for interested parties?
* What other data should be collected?
* How often would the model need to be updated, etc.?

In [19]:
# Kevin insert work here

# <a name="Exceptional-Work"></a>Exceptional Work
*You have free reign to provide additional analyses or combine analyses.*